# Wall Roughness

In this example we will set up and run simulation of an Onera M6 wing in two different configurations. One will have smooth wall boundary condition while the other one will have wall bondary condition with set roughness height. We will then compare the two using report functionality.

## 1. Importing dependencies

First we import flow360 as well as `u` namespace and `OM6Wing` example assets for convenience.

In [1]:
import flow360 as fl
from flow360.component.simulation.unit_system import u
from flow360.examples import OM6wing

## 2. Project Creation

A Flow360 `Project` is a container for simulation assets. Here, we'll create a project from a pre-existing volume mesh file for the OM6 wing. We also download example's asset files.

In [2]:
OM6wing.get_files()

project = fl.Project.from_volume_mesh(OM6wing.mesh_filename, name="Wall roughness")

vm = project.volume_mesh

[14:11:09] INFO: VolumeMesh successfully submitted:                                                                
                   type   = Volume Mesh                                                                            
                   name   = Wall roughness                                                                         
                   id     = vm-b0693406-b929-4b69-bcf0-ed091a8e338d                                                
                   status = uploaded                                                                               
           

## 3. Simulation Parameters

This section defines the simulation parameters. This includes defining the operating conditions, time-stepping parameters, and the physical models for the simulation.


In [3]:
with fl.SI_unit_system:
    params = fl.SimulationParams(
        reference_geometry=fl.ReferenceGeometry(
            area=1.15315084119231,
            moment_center=[0, 0, 0],
            moment_length=[1.47602, 0.801672958512342, 1.47602],
        ),
        operating_condition=fl.AerospaceCondition(velocity_magnitude=286, alpha=10 * fl.u.deg),
        time_stepping=fl.Steady(max_steps=2000),
        outputs=[
            fl.SliceOutput(
                entities=[
                    *[
                        fl.Slice(
                            name=f"slice_y_{name}",
                            normal=(0, 1, 0),
                            origin=(0, y, 0),
                        )
                        for name, y in zip(
                            ["0_2", "0_4", "0_6", "0_8"], [0.2, 0.4, 0.6, 0.8]
                        )
                    ]
                ],
                output_fields=["velocity", "velocity_x", "velocity_y", "velocity_z", "Cp"],
            ),
        ],
    )

[14:11:26] INFO: using: SI unit system for unit inference.

### Wall Models and Roughness Height

Here we define the boundary conditions for the simulation. We use `fl.Wall` for the wing surface, `fl.SlipWall` for the symmetry plane, and `fl.Freestream` for the farfield.

For the wing surface, we specify a `roughness_height`. This parameter models the effect of surface roughness on the flow, which can have significant impact on the aerodynamic performance. 

Physically it represents the equivalent uniform sand-grain roughness height ($k_s$), which is equal to the grain's diameter. A positive value of `roughness_height` will lower near wall velocity.

We will set it here to 0.1 mm, which is within the typical range of roughness heights for a wing.

In [4]:
with fl.SI_unit_system:
    params.models = [
        fl.Wall(
            surfaces=vm["1"],
            roughness_height=1e-4 * fl.u.m
        ),
        fl.SlipWall(surfaces=vm["2"]),
        fl.Freestream(surfaces=vm["3"]),
    ]

           INFO: using: SI unit system for unit inference.

## 4. Executing the Simulations

We will run two simulations to compare the effects of wall roughness.
1. The first case is run with the `roughness_height` defined above.
2. For the second case, we modify the `roughness_height` to zero to simulate a smooth wall, and then run the simulation.

The cases are run in parallel on the cloud. We then wait for both simulations to complete.


In [5]:
case_1 = project.run_case(params, "With wall roughness")

params.models[0].roughness_height=0 * fl.u.m

case_2 = project.run_case(params, "Without wall roughness")

cases = [case_1, case_2]

for case in cases:
    case.wait()

           INFO: using: SI unit system for unit inference.

[14:11:28] INFO: Successfully submitted:                                                                           
                   type   = Case                                                                                   
                   name   = With wall roughness                                                                    
                   id     = case-be0f88f0-cc06-42ed-ba7f-27bee6ea8b45                                              
                   status = pending                                                                                
           

[14:11:29] INFO: using: SI unit system for unit inference.

[14:11:31] INFO: Successfully submitted:                                                                           
                   type   = Case                                                                                   
                   name   = Without wall roughness                                                                 
                   id     = case-3aa51188-c88e-4c57-ba74-a79fbec91c0e                                              
                   status = pending                                                                                
           

## 5. Report Generation

To easily compare the results, we will generate a report. It will include:
- A table with data for drag (CD) and lift (CL) coefficients, including the delta between the two cases.
- 3D visualizations of velocity slice.
- 3D visualizations of surface friction vectors (CfVec) using Line Integral Convolution (LIC)

In [6]:
avg = fl.report.Average(fraction=0.1)
CD = fl.report.DataItem(data="surface_forces/totalCD", title="CD", operations=avg)
CL = fl.report.DataItem(data="surface_forces/totalCL", title="CL", operations=avg)

side_camera = fl.report.Camera(
    position=(0, -1, 0),
    look_at=(0, 0, 0),
    up=(0, 0, 1),
    pan_target=(0.55, 0, 0.0),
    dimension=1.2,
    dimension_dir="width"
)
top_camera = fl.report.TopCamera(
    pan_target=(0.55, 0.75, 0.0),
    dimension=1.5,
    dimension_dir="height"
)

coefficients_table = fl.report.Table(
    data=[CD, fl.report.Delta(data=CD), CL, fl.report.Delta(data=CL)],
    section_title="Statistical data",
)

y_slice = fl.report.Chart3D(
    section_title="Velocity Slice at y=0.2m",
    items_in_row=2,
    force_new_page=True,
    show="slices",
    include=["slice_y_0_2"],
    field="velocity",
    limits=(100 * u.m / u.s, 450 * u.m / u.s),
    camera=side_camera,
    fig_name="slice_y_0_2",
)

surface_lic = fl.report.Chart3D(
    section_title="Surface CfVec",
    items_in_row=2,
    force_new_page=True,
    show="boundaries",
    field="CfVec",
    mode="lic",
    limits=(0, 0.013),
    camera=top_camera,
    fig_name="surface_lic",
)

### Subbmiting the report

All that is left is to finalize the template for our report and submit it.

In [7]:
report = fl.report.ReportTemplate(
    title="Wall roughness comparison",
    items=[
        coefficients_table,
        y_slice,
        surface_lic,
    ],
    settings=fl.report.Settings(dpi=150),
)

report = report.create_in_cloud(
    "Wall roughness comparison",
    cases,
    solver_version="release-25.7.1",
)

report.wait()
report.download("report.pdf")

[14:13:30] INFO: Saved to report.pdf

'report.pdf'